# Paint your dreams with Intel Optimum OpenVINO and Latent Consistency Models 

Install required packages

In [1]:
#%pip install -q "openvino>=2023.3.0"
%pip install -q "onnx>=1.11.0"
%pip install -q "optimum-intel[diffusers]@git+https://github.com/huggingface/optimum-intel.git" "ipywidgets" "torch>=2.1" "transformers>=4.33.0" --extra-index-url https://download.pytorch.org/whl/cpu
%pip install -q "gradio"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import warnings

warnings.filterwarnings("ignore")

Running inference using Optimum Intel OVLatentConsistencyModelPipeline


In [2]:
from optimum.intel.openvino import OVLatentConsistencyModelPipeline
from pathlib import Path

if not Path("./openvino_ir").exists():
    ov_pipeline = OVLatentConsistencyModelPipeline.from_pretrained("SimianLuo/LCM_Dreamshaper_v7", height=512, width=512, export=True, compile=False)
    ov_pipeline.save_pretrained("./openvino_ir")
else:
    ov_pipeline = OVLatentConsistencyModelPipeline.from_pretrained("./openvino_ir", export=False, compile=False)

ov_pipeline.reshape(batch_size=1, height=512, width=512, num_images_per_prompt=1)


Framework not specified. Using pt to export the model.
Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
Keyword arguments {'subfolder': '', 'trust_remote_code': False} are not expected by StableDiffusionPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Using framework PyTorch: 2.3.0+cpu
Using framework PyTorch: 2.3.0+cpu
Using framework PyTorch: 2.3.0+cpu
Using framework PyTorch: 2.3.0+cpu


OVLatentConsistencyModelPipeline {
  "_class_name": "OVLatentConsistencyModelPipeline",
  "_diffusers_version": "0.27.2",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "LCMScheduler"
  ],
  "text_encoder": [
    "optimum",
    "OVModelTextEncoder"
  ],
  "text_encoder_2": [
    null,
    null
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "optimum",
    "OVModelUnet"
  ],
  "vae_decoder": [
    "optimum",
    "OVModelVaeDecoder"
  ],
  "vae_encoder": [
    "optimum",
    "OVModelVaeEncoder"
  ]
}

In [3]:
ov_pipeline.to("GPU.0")
ov_pipeline.compile()

Compiling the vae_decoder to GPU.0 ...
Compiling the unet to GPU.0 ...
Compiling the text_encoder to GPU.0 ...
Compiling the vae_encoder to GPU.0 ...


Run AI inference on the integrated GPU with gradio

In [4]:
import gradio as gr
import time
generate_images = True

examples = [
    "portrait photo of a girl, photograph, highly detailed face, depth of field, moody light, golden hour,"
    "style by Dan Winters, Russell James, Steve McCurry, centered, extremely detailed, Nikon D850, award winning photography",
    "Self-portrait oil painting, a beautiful cyborg with golden hair, 8k",
    "Astronaut in a jungle, cold color palette, muted colors, detailed, 8k",
    "A photo of beautiful mountain with realistic sunset and blue lake, highly detailed, masterpiece",
]


# Function to toggle the flag for image generation
def stop_generation():
    global generate_images
    generate_images = False

def workerLCM(prompt: str):
    global generate_images
    generate_images = True
    while True:
        if not generate_images:       
            break 
        start_time = time.time()
        result = ov_pipeline(prompt=prompt, num_inference_steps=4, guidance_scale=8.0, height=512, width=512).images[0]
        end_time = time.time()
        processing_time = end_time - start_time
        yield  result,round(processing_time, 5)

with gr.Blocks() as demo:
    with gr.Group():
        with gr.Row():
            prompt = gr.Text(
                value=examples[0],
                label="Prompt",
                max_lines=1,
                placeholder="Enter your prompt here",
            )
        with gr.Row() as imagerow:
            gr.Column(scale=1)
            with gr.Column(scale=2) as gpu_column:
                result = gr.Image(label="Generated image", elem_id="output_image")
                result_time_label = gr.Text("", label="Processing Time", type="text")
            gr.Column(scale=1)
        with gr.Row() as image_button:
                run_button = gr.Button("Start image generation")
                stop_button = gr.Button("Stop image generation")
               
    gr.Examples(
        examples=examples,
        inputs=prompt,
        outputs=result,
        cache_examples=False,
    )
    gr.on(
        triggers=[
            prompt.submit,
            run_button.click,
        ],
        fn=workerLCM,
        inputs=[
            prompt
        ],
        outputs=[
            result,result_time_label
        ],
    )
    gr.on(triggers=[stop_button.click], fn=stop_generation)


demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]